# Inital Exploration & Feature Engineering

This notebook focusses on inital exploration of the dataset, exploring the nested dataframes, and then feature engineering. The exported feature dataframe is explored further in the following notebook.

In this notebook you will find the following steps taken:
- Initial Exploration
    - Read in the data
    - Initial exploration of the dataset
    - Explore the nested dataframes individually
    - Get an understanding of columns, making note of possible features to create/explore
- Feature Engineering
    - Create dataframe of features based on initial exploration
    - Export feature dataframe to csv

## Imports

In [1]:
import pandas as pd
import json
from functools import reduce
from pathlib import Path

In [2]:
ROOT_DIR = Path().resolve().parent
DATA_DIR = ROOT_DIR / "data"

## Reading & Exploring Data

In [3]:
df = pd.read_json(DATA_DIR / 'customers.json', lines=True)
df.head(5)

,fraudulent,customer,orders,paymentMethods,transactions
0,False,"{'customerEmail': 'josephhoward@yahoo.com', 'c...","[{'orderId': 'vjbdvd', 'orderAmount': 18, 'ord...","[{'paymentMethodId': 'wt07xm68b', 'paymentMeth...","[{'transactionId': 'a9lcj51r', 'orderId': 'vjb..."
1,True,"{'customerEmail': 'evansjeffery@yahoo.com', 'c...","[{'orderId': 'nlghpa', 'orderAmount': 45, 'ord...","[{'paymentMethodId': 'y3xp697jx', 'paymentMeth...","[{'transactionId': '5mi94sfw', 'orderId': 'nlg..."
2,False,"{'customerEmail': 'andersonwilliam@yahoo.com',...","[{'orderId': 'yk34y2', 'orderAmount': 33, 'ord...","[{'paymentMethodId': '8pneoi03z', 'paymentMeth...","[{'transactionId': 'q3lyvbza', 'orderId': 'yk3..."
3,False,"{'customerEmail': 'rubenjuarez@yahoo.com', 'cu...","[{'orderId': 'fbz9ep', 'orderAmount': 34, 'ord...","[{'paymentMethodId': 'pdxjdwui4', 'paymentMeth...","[{'transactionId': 'vx4cjc27', 'orderId': 'fbz..."
4,True,"{'customerEmail': 'uchen@malone.com', 'custome...","[{'orderId': '56h7iw', 'orderAmount': 71, 'ord...","[{'paymentMethodId': 'w1i1zq3rg', 'paymentMeth...","[{'transactionId': 'q8j3dgni', 'orderId': '56h..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   fraudulent      168 non-null    bool  
 1   customer        168 non-null    object
 2   orders          168 non-null    object
 3   paymentMethods  168 non-null    object
 4   transactions    168 non-null    object
dtypes: bool(1), object(4)
memory usage: 5.5+ KB


In [5]:
df.describe()

,fraudulent,customer,orders,paymentMethods,transactions
count,168,168,168,168,168
unique,2,168,142,157,142
top,False,"{'customerEmail': 'josephhoward@yahoo.com', 'c...",[],[],[]
freq,107,1,27,12,27


In [6]:
display(df['fraudulent'].value_counts())
print(f'\nNot Fraudulent: {round(df['fraudulent'].value_counts().iloc[0] / len(df) * 100,2)}%')
print(f'Fraudulent: {round(df['fraudulent'].value_counts().iloc[1] / len(df) * 100,2)}%')

fraudulent
False    107
True      61
Name: count, dtype: int64


Not Fraudulent: 63.69%
Fraudulent: 36.31%


#### Key Takeaways

- The dataset contains 168 records, with the `fraudulent` label showing class imbalance (64% labelled as non-fraud).
- Columns `customer`, `orders`, `paymentMethods`, and `transactions` are nested structures and require flattening/feature extraction.
- Each customer appears to be unique.
- There are missing values / empty structures in `orders`, `paymentMethods`, and `transactions`.
- Feature engineering is required to extract meaningful numerical inputs for modelling.

## Exploring Embedded DataFrames
Since columns `customer`, `orders`, `paymentMethods`, and `transactions` are nested structures, the following section explores each individually.

### Customers

In [7]:
# Expand customers whilst maintaining fraudulent label
customer_df = pd.concat([df['fraudulent'], pd.json_normalize(df['customer'])], axis=1)
customer_df.head(5)

,fraudulent,customerEmail,customerPhone,customerDevice,customerIPAddress,customerBillingAddress
0,False,josephhoward@yahoo.com,400-108-5415,yyeiaxpltf82440jnb3v,8.129.104.40,"5493 Jones Islands\nBrownside, CA 51896"
1,True,evansjeffery@yahoo.com,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...
2,False,andersonwilliam@yahoo.com,024.420.0375,4m7h5ipl1shyavt6vv2r,67b7:3db8:67e0:3bea:b9d0:90c1:2b60:b9f0,"8478 Sean Ridges Apt. 441\nDavisberg, PR 72250"
3,False,rubenjuarez@yahoo.com,670.664.8168x94985,slovx60t0i558may4ks0,95de:8565:5a66:792c:26e0:6cfb:7d87:11af,"7769 Elizabeth Bridge Apt. 343\nNortonstad, FM..."
4,True,uchen@malone.com,1-981-877-0870,j0pd24k5h8dl2fqu0cz4,196.89.235.192,"148 Russell Lodge Apt. 445\nPort Jenniferside,..."


In [8]:
customer_df.describe()

,fraudulent,customerEmail,customerPhone,customerDevice,customerIPAddress,customerBillingAddress
count,168,168,168,168,168,168
unique,2,161,168,168,165,166
top,False,johnlowery@gmail.com,400-108-5415,yyeiaxpltf82440jnb3v,45.203.99.249,49680 Brian Squares Apt. 122\nPort Walterburgh...
freq,107,8,1,1,4,3


In [9]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   fraudulent              168 non-null    bool  
 1   customerEmail           168 non-null    object
 2   customerPhone           168 non-null    object
 3   customerDevice          168 non-null    object
 4   customerIPAddress       168 non-null    object
 5   customerBillingAddress  168 non-null    object
dtypes: bool(1), object(5)
memory usage: 6.9+ KB


#### Key Takeaways
- This dataframe consists of customer details
- Although these cannot be used directly in the modelling process, they hold potential to extract key insights during feature extraction
- Initial thoughts:
    - multiple email addresses, IP addresses & billing addresses are seen across multiple customers
        - email 'johnlowery@gmail.com' appears 8 times
        - address '49680 Brian Squares Apt. 122\nPort Walterburgh...' appears 3 times
        - possible feature: duplicated email / IP addresses (although people can share) / billing address (also shareable)
    - possible invalid email addresses or uncommon domains
    - possible invalid IP addresses
    - possible invalid phone numbers

### Orders

In [10]:
def explode_column_to_dataframe(df, column_name: str) -> pd.DataFrame:
    """
    Converts a column of nested lists of dictionaries into a flat DataFrame.

    Explodes the specified column, flattens nested data, assigns 'customer_id' from the index,
    and retains the 'fraudulent' label.
    """
    exploded_df = df[['fraudulent', column_name]]
    exploded_df = exploded_df.explode(column_name).reset_index().rename(columns={'index':'customer_id'})
    exploded_df = pd.json_normalize(json.loads(exploded_df.to_json(orient='records'))).drop(columns=[column_name]).dropna()
    exploded_df.columns = [col.split('.')[-1] for col in exploded_df.columns]
    return exploded_df

In [11]:
orders_df = explode_column_to_dataframe(df, column_name='orders')
orders_df.head(5)

,customer_id,fraudulent,orderId,orderAmount,orderState,orderShippingAddress
0,0,False,vjbdvd,18.0,pending,"5493 Jones Islands\nBrownside, CA 51896"
1,0,False,yp6x27,26.0,fulfilled,"5493 Jones Islands\nBrownside, CA 51896"
2,1,True,nlghpa,45.0,fulfilled,"898 Henry Ports\nNew Keithview, CA 95893-2497"
3,1,True,uw0eeb,23.0,fulfilled,356 Elizabeth Inlet Suite 120\nPort Joshuabury...
4,1,True,bn44oh,43.0,fulfilled,"5093 Bryan Forks\nJoshuaton, FM 01565-9801"


In [12]:
orders_df.describe(include=['O'])

,orderId,orderState,orderShippingAddress
count,478,478,478
unique,478,3,274
top,5lmvrj,fulfilled,"148 Russell Lodge Apt. 445\nPort Jenniferside,..."
freq,1,400,6


#### Column Analysis

In [13]:
orders_df['orderState'].value_counts()

orderState
fulfilled    400
failed        50
pending       28
Name: count, dtype: int64

In [14]:
orders_df['orderShippingAddress'].value_counts().head(5)

orderShippingAddress
148 Russell Lodge Apt. 445\nPort Jenniferside, TN 60556-0871    6
87096 Warren Turnpike\nHowellview, WA 74727-2420                6
96103 Riley Walks\nMichellefort, ID 98085-6066                  6
87265 Woods Square\nPort Heidimouth, LA 03208-8304              5
904 James Route Suite 595\nLake Ruth, CO 66976                  5
Name: count, dtype: int64

In [15]:
orders_df['orderAmount'].describe()

count    478.000000
mean      35.606695
std       21.667298
min       10.000000
25%       22.000000
50%       35.000000
75%       46.000000
max      353.000000
Name: orderAmount, dtype: float64

#### Billing & Shipping Addresses

In [16]:
# Merge to get customer billing address
orders_df = pd.merge(orders_df, customer_df['customerBillingAddress'], left_on='customer_id', right_on=customer_df.index)

# Orders with different billing to shipping address
orders_df[['fraudulent','orderShippingAddress', 'customerBillingAddress']][orders_df['customerBillingAddress'] != orders_df['orderShippingAddress']].head(10)

,fraudulent,orderShippingAddress,customerBillingAddress
2,True,"898 Henry Ports\nNew Keithview, CA 95893-2497",356 Elizabeth Inlet Suite 120\nPort Joshuabury...
4,True,"5093 Bryan Forks\nJoshuaton, FM 01565-9801",356 Elizabeth Inlet Suite 120\nPort Joshuabury...
7,False,"22396 Victor Lights\nCarsonstad, MO 93914","8478 Sean Ridges Apt. 441\nDavisberg, PR 72250"
10,False,"428 Ricardo Mountain\nPort Lorihaven, UT 07665...","7769 Elizabeth Bridge Apt. 343\nNortonstad, FM..."
14,True,"620 Alexander Views\nMichaelstad, AK 94959-7484","148 Russell Lodge Apt. 445\nPort Jenniferside,..."
18,True,"764 Collier Springs\nJackfurt, NE 95955-5328","65233 Hutchinson Mountains\nMartinezside, OR 7..."
19,True,"9447 Cook Extensions\nRebeccahaven, NY 78148-0238","3960 Emily Fort\nStaceytown, AR 21817-3325"
23,True,"27513 Betty Gardens Suite 901\nStewartport, CA...","484 Pamela Pass\nLake Jessicaview, WI 12942-9074"
27,True,"256 Vang Prairie Suite 748\nWest Troyland, IA ...","27061 Wright Forks\nCraigland, MN 59915"
32,False,"5918 Jeremy Spur Apt. 524\nMartinborough, SC 7...","46383 Matthew Track Apt. 220\nRomanburgh, DC 9..."


In [17]:
orders_df[['fraudulent','orderShippingAddress', 'customerBillingAddress']][orders_df['customerBillingAddress'] != orders_df['orderShippingAddress']]['fraudulent'].value_counts()

fraudulent
True     81
False    69
Name: count, dtype: int64

#### Key Takeaways
- Details on orders per customer
- 3 order states: ['pending', 'fulfilled', 'failed']
    - 400/478 fulfilled
    - 50 failed
    - 28 pending
- Order amount
    - avg: 35
    - min: 10
    - max: 353 - (far of average, anomaly?)
- A customer can have multiple shipping addresses
- A customer can have shipping address different to billing address
    - This doesn't appear to be clear indiciation of fraud

### Payments Methods

In [18]:
payments_df = explode_column_to_dataframe(df, column_name='paymentMethods')
payments_df.head(5)

,customer_id,fraudulent,paymentMethodId,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer
0,0,False,wt07xm68b,True,card,JCB 16 digit,Citizens First Banks
1,1,True,y3xp697jx,True,bitcoin,VISA 16 digit,Solace Banks
2,1,True,6krszxc05,False,card,VISA 16 digit,Vertex Bancorp
3,1,True,5z1szj2he,False,card,Diners Club / Carte Blanche,His Majesty Bank Corp.
4,1,True,m52tx8e1s,False,card,Mastercard,Vertex Bancorp


In [19]:
payments_df.describe(include=['O'])

,paymentMethodId,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer
count,342,342,342,342,342
unique,342,2,4,10,20
top,rh7676yct,False,card,JCB 16 digit,Her Majesty Trust
freq,1,302,242,65,43


#### Column Analysis

In [20]:
payments_df['paymentMethodRegistrationFailure'].value_counts()

paymentMethodRegistrationFailure
False    302
True      40
Name: count, dtype: int64

In [21]:
payments_df['paymentMethodType'].value_counts()

paymentMethodType
card         242
apple pay     36
paypal        36
bitcoin       28
Name: count, dtype: int64

In [22]:
payments_df['paymentMethodProvider'].value_counts()

paymentMethodProvider
JCB 16 digit                   65
VISA 16 digit                  57
Voyager                        36
Diners Club / Carte Blanche    34
VISA 13 digit                  32
Maestro                        32
Discover                       25
American Express               22
JCB 15 digit                   20
Mastercard                     19
Name: count, dtype: int64

In [23]:
payments_df['paymentMethodIssuer'].value_counts()

paymentMethodIssuer
Her Majesty Trust           43
Vertex Bancorp              37
Fountain Financial Inc.     35
His Majesty Bank Corp.      33
Bastion Banks               29
Bulwark Trust Corp.         29
Citizens First Banks        28
Solace Banks                27
Grand Credit Corporation    27
Rose Bancshares             25
B                            7
e                            5
c                            4
r                            3
n                            2
x                            2
o                            2
                             2
p                            1
a                            1
Name: count, dtype: int64

#### Key Takeaways
- Details on payments methods and details per customer, across all orders
- 40/342 payment method registration failure
- 4 payment method types: ['card', 'apple pay', 'paypal', 'bitcoin']
    - Card payments were the most common method, accounting for the majority of transactions (242)
- 10 unique payment method providers
- 20 payment method issuer values
    - Some unusual values with low value counts, e.g. ['B', 'p', 'x', 'e', ...]

### Transactions

In [24]:
transactions_df = explode_column_to_dataframe(df, column_name='transactions')
transactions_df.head(5)

,customer_id,fraudulent,transactionId,orderId,paymentMethodId,transactionAmount,transactionFailed
0,0,False,a9lcj51r,vjbdvd,wt07xm68b,18.0,False
1,0,False,y4wcv03i,yp6x27,wt07xm68b,26.0,False
2,1,True,5mi94sfw,nlghpa,41ug157xz,45.0,False
3,1,True,br8ba1nu,uw0eeb,41ug157xz,23.0,False
4,1,True,a33145ss,bn44oh,y3xp697jx,43.0,True


In [25]:
transactions_df.describe(include=['O'])

,transactionId,orderId,paymentMethodId,transactionFailed
count,623,623,623,623
unique,623,478,237,2
top,wgmajf82,93lzyc,4nf29mjz1,False
freq,1,7,15,455


In [26]:
transactions_df['paymentMethodId'].value_counts().head(10)

paymentMethodId
4nf29mjz1    15
w8f4lcsen    13
pube8iuin    11
syeg4jsea    10
r0735q7at     8
l1qil6tjh     7
opyep1aoj     7
88huac0bv     6
5n1znn6g7     6
wejtuspvr     6
Name: count, dtype: int64

#### Failed transactions

In [27]:
transactions_df['transactionFailed'].value_counts()

transactionFailed
False    455
True     168
Name: count, dtype: int64

#### Number of transactions per order

In [28]:
transactions_counts = transactions_df.groupby(['customer_id', 'orderId'])['transactionId'].nunique().reset_index()
transactions_counts.rename(columns={'transactionId': 'num_transactions_per_order'}, inplace=True)
transactions_counts.sort_values(by='num_transactions_per_order', ascending=False).head(5)

,customer_id,orderId,num_transactions_per_order
398,143,93lzyc,7
399,143,bunmsb,6
422,151,st9dv0,5
113,42,v1sq5r,5
367,132,fng05a,4


#### Number of payment methods per customer

In [29]:
payment_counts = transactions_df.groupby('customer_id')['paymentMethodId'].nunique().reset_index()
payment_counts.rename(columns={'paymentMethodId': 'num_payment_methods'}, inplace=True)
payment_counts['num_payment_methods'].value_counts()

num_payment_methods
1    76
2    44
3    14
4     4
5     3
Name: count, dtype: int64

#### Number of payment methods per customer per order

In [30]:
payment_counts = transactions_df.groupby(['customer_id', 'orderId'])['paymentMethodId'].nunique().reset_index()
payment_counts.rename(columns={'paymentMethodId': 'num_payment_methods'}, inplace=True)
payment_counts['num_payment_methods'].value_counts()

num_payment_methods
1    435
2     43
Name: count, dtype: int64

#### Key Takeaways
- Details on transactions per customer per order
- 168/623 Failed transactions
    - Potential indication of fraud?
- Multiple transactions per order
    - Top number of transactions per orderID is 7
- Multiple payment methods per customer
- Multiple payment methods per order

## Feature Engineering
This section focusses on extracting insights from each embedded dataframe, creating features for modelling.

### Customers

In [31]:
customer_df.head(3)

,fraudulent,customerEmail,customerPhone,customerDevice,customerIPAddress,customerBillingAddress
0,False,josephhoward@yahoo.com,400-108-5415,yyeiaxpltf82440jnb3v,8.129.104.40,"5493 Jones Islands\nBrownside, CA 51896"
1,True,evansjeffery@yahoo.com,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...
2,False,andersonwilliam@yahoo.com,024.420.0375,4m7h5ipl1shyavt6vv2r,67b7:3db8:67e0:3bea:b9d0:90c1:2b60:b9f0,"8478 Sean Ridges Apt. 441\nDavisberg, PR 72250"


#### Valid email address

In [32]:
# Regex pattern to check for a valid email
email_pattern = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'

# Label if each email is valid
customer_df['customer_valid_email'] = customer_df['customerEmail'].str.match(email_pattern)
customer_df.head(5)

,fraudulent,customerEmail,customerPhone,customerDevice,customerIPAddress,customerBillingAddress,customer_valid_email
0,False,josephhoward@yahoo.com,400-108-5415,yyeiaxpltf82440jnb3v,8.129.104.40,"5493 Jones Islands\nBrownside, CA 51896",True
1,True,evansjeffery@yahoo.com,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,True
2,False,andersonwilliam@yahoo.com,024.420.0375,4m7h5ipl1shyavt6vv2r,67b7:3db8:67e0:3bea:b9d0:90c1:2b60:b9f0,"8478 Sean Ridges Apt. 441\nDavisberg, PR 72250",True
3,False,rubenjuarez@yahoo.com,670.664.8168x94985,slovx60t0i558may4ks0,95de:8565:5a66:792c:26e0:6cfb:7d87:11af,"7769 Elizabeth Bridge Apt. 343\nNortonstad, FM...",True
4,True,uchen@malone.com,1-981-877-0870,j0pd24k5h8dl2fqu0cz4,196.89.235.192,"148 Russell Lodge Apt. 445\nPort Jenniferside,...",True


### Orders

In [33]:
orders_df.head(5)

,customer_id,fraudulent,orderId,orderAmount,orderState,orderShippingAddress,customerBillingAddress
0,0,False,vjbdvd,18.0,pending,"5493 Jones Islands\nBrownside, CA 51896","5493 Jones Islands\nBrownside, CA 51896"
1,0,False,yp6x27,26.0,fulfilled,"5493 Jones Islands\nBrownside, CA 51896","5493 Jones Islands\nBrownside, CA 51896"
2,1,True,nlghpa,45.0,fulfilled,"898 Henry Ports\nNew Keithview, CA 95893-2497",356 Elizabeth Inlet Suite 120\nPort Joshuabury...
3,1,True,uw0eeb,23.0,fulfilled,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,356 Elizabeth Inlet Suite 120\nPort Joshuabury...
4,1,True,bn44oh,43.0,fulfilled,"5093 Bryan Forks\nJoshuaton, FM 01565-9801",356 Elizabeth Inlet Suite 120\nPort Joshuabury...


#### Order State & Count

In [34]:
order_counts_df = orders_df.groupby('customer_id')['orderState'].value_counts().unstack(fill_value=0)
order_counts_df.columns = ['fulfilled_count', 'failed_count', 'pending_count']
order_counts_df['total_count'] = order_counts_df.sum(axis=1)

order_counts_df.head(5)

,fulfilled_count,failed_count,pending_count,total_count
customer_id,,,,
0,0,1,1,2
1,0,3,0,3
2,1,2,0,3
3,0,3,0,3
4,1,6,0,7


#### Order Amounts

In [35]:
order_amounts_agg_df = orders_df.groupby('customer_id')['orderAmount'].agg(
    sum_amount='sum',
    mean_amount='mean',
    max_amount='max',
    min_amount='min'
).reset_index()

order_amounts_agg_df.head(5)

,customer_id,sum_amount,mean_amount,max_amount,min_amount
0,0,44.0,22.000000,26.0,18.0
1,1,111.0,37.000000,45.0,23.0
2,2,82.0,27.333333,33.0,24.0
3,3,85.0,28.333333,34.0,25.0
4,4,411.0,58.714286,71.0,32.0


#### Ratio of number of shipping addresses across orders to total orders

In [36]:
shipping_address_count_df = orders_df.groupby('customer_id')['orderShippingAddress'].nunique().reset_index()
shipping_address_count_df.columns = ['customer_id', 'unique_shipping_address_count']

shipping_ratio_df = pd.merge(shipping_address_count_df, order_counts_df['total_count'], on='customer_id')

# Calculate ratio
shipping_ratio_df['unique_shipping_address_to_order_ratio'] = (
    shipping_ratio_df['unique_shipping_address_count'] / shipping_ratio_df['total_count']
)
shipping_ratio_df = shipping_ratio_df.drop(columns=['unique_shipping_address_count','total_count'])
shipping_ratio_df.head(5)

,customer_id,unique_shipping_address_to_order_ratio
0,0,0.500000
1,1,1.000000
2,2,0.666667
3,3,0.666667
4,4,0.285714


#### Percentage of orders with same billing address & shipping address

In [37]:
orders_df['sameBillingShippingAddress'] = orders_df['customerBillingAddress'] == orders_df['orderShippingAddress']
same_address_percentage_df = orders_df.groupby('customer_id')['sameBillingShippingAddress'].mean().reset_index()
same_address_percentage_df.columns = ['customer_id', 'same_billing_shipping_address_perc']
same_address_percentage_df

,customer_id,same_billing_shipping_address_perc
0,0,1.000000
1,1,0.333333
2,2,0.666667
3,3,0.666667
4,4,0.857143
...,...,...
136,163,0.666667
137,164,0.800000
138,165,0.500000
139,166,0.500000


#### Combine
Combine features to create feature dataframe, to be combined into the final dataframe later on

In [38]:
dfs = [order_counts_df, order_amounts_agg_df, shipping_address_count_df, shipping_ratio_df, same_address_percentage_df]
orders_features_df = reduce(lambda left, right: pd.merge(left, right, on='customer_id', how='inner'), dfs)
orders_features_df

,customer_id,fulfilled_count,failed_count,pending_count,total_count,sum_amount,mean_amount,max_amount,min_amount,unique_shipping_address_count,unique_shipping_address_to_order_ratio,same_billing_shipping_address_perc
0,0,0,1,1,2,44.0,22.000000,26.0,18.0,1,0.500000,1.000000
1,1,0,3,0,3,111.0,37.000000,45.0,23.0,3,1.000000,0.333333
2,2,1,2,0,3,82.0,27.333333,33.0,24.0,2,0.666667,0.666667
3,3,0,3,0,3,85.0,28.333333,34.0,25.0,2,0.666667,0.666667
4,4,1,6,0,7,411.0,58.714286,71.0,32.0,2,0.285714,0.857143
...,...,...,...,...,...,...,...,...,...,...,...,...
136,163,1,2,0,3,126.0,42.000000,61.0,31.0,2,0.666667,0.666667
137,164,0,5,0,5,135.0,27.000000,43.0,14.0,2,0.400000,0.800000
138,165,0,4,0,4,121.0,30.250000,46.0,13.0,3,0.750000,0.500000
139,166,1,5,0,6,287.0,47.833333,64.0,37.0,4,0.666667,0.500000


### Payment Methods

In [39]:
payments_df.head(5)

,customer_id,fraudulent,paymentMethodId,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer
0,0,False,wt07xm68b,True,card,JCB 16 digit,Citizens First Banks
1,1,True,y3xp697jx,True,bitcoin,VISA 16 digit,Solace Banks
2,1,True,6krszxc05,False,card,VISA 16 digit,Vertex Bancorp
3,1,True,5z1szj2he,False,card,Diners Club / Carte Blanche,His Majesty Bank Corp.
4,1,True,m52tx8e1s,False,card,Mastercard,Vertex Bancorp


#### Number of failed payments

In [40]:
payments_df['failed_payment_int'] = payments_df['paymentMethodRegistrationFailure'].astype(int)
failed_payments_count_df = payments_df.groupby('customer_id')['failed_payment_int'].sum().reset_index()
failed_payments_count_df.columns = ['customer_id', 'failed_payment_method_count']
failed_payments_count_df['failed_payment_method_count'].value_counts()

failed_payment_method_count
0    123
1     29
2      2
3      1
4      1
Name: count, dtype: int64

#### Payment method type

In [41]:
payment_method_type_count_df = pd.get_dummies(payments_df['paymentMethodType'], prefix='payment_method').groupby(payments_df['customer_id']).sum()
payment_method_type_count_df.columns = [col + '_count' for col in payment_method_type_count_df.columns]
payment_method_type_count_df = payment_method_type_count_df.reset_index()
payment_method_type_count_df.head(5)

,customer_id,payment_method_apple pay_count,payment_method_bitcoin_count,payment_method_card_count,payment_method_paypal_count
0,0,0,0,1,0
1,1,0,2,5,0
2,2,0,0,2,0
3,3,0,0,1,0
4,4,3,0,3,0


#### Number of unique payment method providers

In [42]:
unique_payment_method_providers_count_df = payments_df.groupby('customer_id')['paymentMethodProvider'].nunique().reset_index()
unique_payment_method_providers_count_df.columns = ['customer_id', 'unique_payment_method_provider_count']
unique_payment_method_providers_count_df

,customer_id,unique_payment_method_provider_count
0,0,1
1,1,6
2,2,2
3,3,1
4,4,4
...,...,...
151,163,1
152,164,2
153,165,1
154,166,6


#### Number of unique payment method issuers

In [43]:
unique_payment_methods_issuers_count_df = payments_df.groupby('customer_id')['paymentMethodIssuer'].nunique().reset_index()
unique_payment_methods_issuers_count_df.columns = ['customer_id', 'unique_payment_method_issuer_count']
unique_payment_methods_issuers_count_df

,customer_id,unique_payment_method_issuer_count
0,0,1
1,1,6
2,2,2
3,3,1
4,4,4
...,...,...
151,163,1
152,164,3
153,165,2
154,166,5


#### Recognised payment method issuers ratio
Unusual values for payment method issuers were highlighted in the inital epxloration.

Create feature for the ratio of payments with unrecognised issuers to total payments per customer

In [44]:
payments_df['paymentMethodIssuer'].unique()

array(['Citizens First Banks', 'Solace Banks', 'Vertex Bancorp',
       'His Majesty Bank Corp.', 'Bastion Banks', 'Her Majesty Trust',
       'Fountain Financial Inc.', 'Grand Credit Corporation', 'B', 'p',
       'x', 'e', 'Bulwark Trust Corp.', 'c', 'a', 'Rose Bancshares', ' ',
       'r', 'n', 'o'], dtype=object)

In [45]:
recognised_payment_method_issuer_list = [
    "Her Majesty Trust",
    "Vertex Bancorp",
    "Fountain Financial Inc.",
    "His Majesty Bank Corp.",
    "Bastion Banks",
    "Bulwark Trust Corp.",
    "Citizens First Banks",
    "Solace Banks",
    "Grand Credit Corporation",
    "Rose Bancshares"
]

# Ratio of payments with unrecognised issuers to total payments per customer
payments_df['recognised_payment_method_issuer_ratio'] = payments_df['paymentMethodIssuer'].isin(recognised_payment_method_issuer_list)
recognised_payment_issuer_df = payments_df.groupby('customer_id')['recognised_payment_method_issuer_ratio'].mean().reset_index()
recognised_payment_issuer_df['recognised_payment_method_issuer_ratio'].value_counts()

recognised_payment_method_issuer_ratio
1.000000    140
0.000000     15
0.333333      1
Name: count, dtype: int64

#### Combine
Combine features into one feature dataframe, specific to payments

In [46]:
dfs = [
    failed_payments_count_df, 
    payment_method_type_count_df, 
    unique_payment_method_providers_count_df, 
    unique_payment_methods_issuers_count_df, 
    recognised_payment_issuer_df
]
payments_features_df = reduce(lambda left, right: pd.merge(left, right, on='customer_id', how='inner'), dfs)
payments_features_df.head(5)

,customer_id,failed_payment_method_count,payment_method_apple pay_count,payment_method_bitcoin_count,payment_method_card_count,payment_method_paypal_count,unique_payment_method_provider_count,unique_payment_method_issuer_count,recognised_payment_method_issuer_ratio
0,0,1,0,0,1,0,1,1,1.0
1,1,1,0,2,5,0,6,6,1.0
2,2,0,0,0,2,0,2,2,1.0
3,3,0,0,0,1,0,1,1,1.0
4,4,0,3,0,3,0,4,4,1.0


### Transactions

In [47]:
transactions_df.head(5)

,customer_id,fraudulent,transactionId,orderId,paymentMethodId,transactionAmount,transactionFailed
0,0,False,a9lcj51r,vjbdvd,wt07xm68b,18.0,False
1,0,False,y4wcv03i,yp6x27,wt07xm68b,26.0,False
2,1,True,5mi94sfw,nlghpa,41ug157xz,45.0,False
3,1,True,br8ba1nu,uw0eeb,41ug157xz,23.0,False
4,1,True,a33145ss,bn44oh,y3xp697jx,43.0,True


#### Number of transactions per customer

In [48]:
transactions_df.groupby(['customer_id'])['transactionId'].count().reset_index()

,customer_id,transactionId
0,0,2
1,1,3
2,2,5
3,3,3
4,4,7
...,...,...
136,163,3
137,164,6
138,165,4
139,166,8


In [49]:
transactions_counts_df = transactions_df.groupby(['customer_id'])['transactionId'].nunique().reset_index()
transactions_counts_df.rename(columns={'transactionId': 'total_transaction_count'}, inplace=True)
transactions_counts_df.head(10)

,customer_id,total_transaction_count
0,0,2
1,1,3
2,2,5
3,3,3
4,4,7
5,5,1
6,6,2
7,7,6
8,8,5
9,10,6


#### Number of failed transactions per customer

In [50]:
transactions_df['transactionFailed'] = transactions_df['transactionFailed'].astype(bool)
failed_transaction_count_df = transactions_df.groupby('customer_id')['transactionFailed'].sum().reset_index()
failed_transaction_count_df.columns = ['customer_id', 'failed_transaction_count']
failed_transaction_count_df.head(10)

,customer_id,failed_transaction_count
0,0,0
1,1,1
2,2,2
3,3,0
4,4,2
5,5,0
6,6,1
7,7,2
8,8,3
9,10,2


In [51]:
failed_transaction_count_df['failed_transaction_count'].value_counts()

failed_transaction_count
0     54
1     42
2     28
3     10
4      3
5      2
6      1
12     1
Name: count, dtype: int64

In [52]:
transaction_stats_df = pd.merge(transactions_counts_df,failed_transaction_count_df, on='customer_id')

# Calculate the failed percentage
transaction_stats_df['failed_percentage'] = transaction_stats_df['failed_transaction_count'] / transaction_stats_df['total_transaction_count']
transaction_stats_df

,customer_id,total_transaction_count,failed_transaction_count,failed_percentage
0,0,2,0,0.000000
1,1,3,1,0.333333
2,2,5,2,0.400000
3,3,3,0,0.000000
4,4,7,2,0.285714
...,...,...,...,...
136,163,3,0,0.000000
137,164,6,1,0.166667
138,165,4,0,0.000000
139,166,8,2,0.250000


#### Avg number of transactions per order per customer

In [53]:
# Get the number of transactions per order for each customer
transactions_counts = transactions_df.groupby(['customer_id', 'orderId'])['transactionId'].nunique().reset_index()
transactions_counts.rename(columns={'transactionId': 'num_transactions'}, inplace=True)

# Calculate  average number of transactions per order for each customer
avg_transactions_per_order = transactions_counts.groupby('customer_id')['num_transactions'].mean().reset_index()
avg_transactions_per_order.rename(columns={'num_transactions': 'avg_transactions_per_order'}, inplace=True)

avg_transactions_per_order

,customer_id,avg_transactions_per_order
0,0,1.000000
1,1,1.000000
2,2,1.666667
3,3,1.000000
4,4,1.000000
...,...,...
136,163,1.000000
137,164,1.200000
138,165,1.000000
139,166,1.333333


In [54]:
avg_transactions_per_order['avg_transactions_per_order'].value_counts()

avg_transactions_per_order
1.000000    62
1.250000    18
1.500000    10
2.000000    10
1.200000    10
1.666667     6
1.333333     6
1.400000     5
1.166667     3
2.500000     2
1.600000     2
2.666667     1
2.333333     1
1.750000     1
1.375000     1
1.625000     1
4.000000     1
5.000000     1
Name: count, dtype: int64

#### Combine

In [55]:
dfs = [
    transactions_counts_df, 
    failed_transaction_count_df, 
    avg_transactions_per_order, 
]
transactions_features_df = reduce(lambda left, right: pd.merge(left, right, on='customer_id', how='inner'), dfs)
transactions_features_df.head(10)

,customer_id,total_transaction_count,failed_transaction_count,avg_transactions_per_order
0,0,2,0,1.000000
1,1,3,1,1.000000
2,2,5,2,1.666667
3,3,3,0,1.000000
4,4,7,2,1.000000
5,5,1,0,1.000000
6,6,2,1,2.000000
7,7,6,2,1.200000
8,8,5,3,2.500000
9,10,6,2,1.200000


### Combined Feature Dataset

In [56]:
def add_prefix(df, prefix):
    """
    Add prefixes to each dataframe, using provided prefix string.
    """
    df.columns = [f'{prefix}{col}' if col != 'customer_id' else col for col in df.columns]
    return df

dfs_with_prefixes = [
    (orders_features_df, 'orders.'),
    (payments_features_df, 'payments.'),
    (transactions_features_df, 'transactions.')
]

# Add prefixes to columns in each dataframe for readability
# E.g. 'max_amount' in orders_features_df -> 'orders.max_amount'
dfs_with_prefixes = [add_prefix(df, prefix) for df, prefix in dfs_with_prefixes]

In [57]:
# Left join each dataframe with customer_df
features_df = customer_df.copy().reset_index().rename(columns={'index':'customer_id'})  # Start with customer_df
for df in dfs_with_prefixes:
    features_df = features_df.merge(df, on='customer_id', how='left')

# Save the result to CSV
features_df.to_csv(DATA_DIR / 'customer_dataset_with_features.csv', index=False)
features_df.head(5)

,customer_id,fraudulent,customerEmail,customerPhone,customerDevice,customerIPAddress,customerBillingAddress,customer_valid_email,orders.fulfilled_count,orders.failed_count,...,payments.payment_method_apple pay_count,payments.payment_method_bitcoin_count,payments.payment_method_card_count,payments.payment_method_paypal_count,payments.unique_payment_method_provider_count,payments.unique_payment_method_issuer_count,payments.recognised_payment_method_issuer_ratio,transactions.total_transaction_count,transactions.failed_transaction_count,transactions.avg_transactions_per_order
0,0,False,josephhoward@yahoo.com,400-108-5415,yyeiaxpltf82440jnb3v,8.129.104.40,"5493 Jones Islands\nBrownside, CA 51896",True,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,2.0,0.0,1.000000
1,1,True,evansjeffery@yahoo.com,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,True,0.0,3.0,...,0.0,2.0,5.0,0.0,6.0,6.0,1.0,3.0,1.0,1.000000
2,2,False,andersonwilliam@yahoo.com,024.420.0375,4m7h5ipl1shyavt6vv2r,67b7:3db8:67e0:3bea:b9d0:90c1:2b60:b9f0,"8478 Sean Ridges Apt. 441\nDavisberg, PR 72250",True,1.0,2.0,...,0.0,0.0,2.0,0.0,2.0,2.0,1.0,5.0,2.0,1.666667
3,3,False,rubenjuarez@yahoo.com,670.664.8168x94985,slovx60t0i558may4ks0,95de:8565:5a66:792c:26e0:6cfb:7d87:11af,"7769 Elizabeth Bridge Apt. 343\nNortonstad, FM...",True,0.0,3.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,3.0,0.0,1.000000
4,4,True,uchen@malone.com,1-981-877-0870,j0pd24k5h8dl2fqu0cz4,196.89.235.192,"148 Russell Lodge Apt. 445\nPort Jenniferside,...",True,1.0,6.0,...,3.0,0.0,3.0,0.0,4.0,4.0,1.0,7.0,2.0,1.000000
